# A WeatherAPI alternative

#### Question 1: Open Meteo or Tomorrow?

I'm going to start with Tomorrow, since the way to request Open Meteo data seems to use a lot of new libraries that look complicated, whereas Tomorrow looks more familiar. Open Meteo also looked too fancy and it was less obvious how to get a simple url

#### Question 2: URL to documentation

I think it's https://docs.tomorrow.io/reference/welcome, but then there are individual pages for different things you can do. The weather forecast doc is: https://docs.tomorrow.io/reference/weather-forecast

#### Question 3: Request for current weather in Urbana, IL

In [134]:
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY_TOMORROW = os.getenv("API_KEY_TOMORROW")

In [135]:
baseurl = 'https://api.tomorrow.io/v4/weather/'

import requests
from prettyprinter import pprint

In [136]:
urbana_current_url = f'{baseurl}realtime?location=61801&apikey={API_KEY_TOMORROW}'

response = requests.get(urbana_current_url)
urbana_current_data = response.json()
pprint(urbana_current_data)

{
    'data': {
        'time': '2025-06-11T18:45:00Z',
        'values': {
            'cloudBase': 16,
            'cloudCeiling': 16,
            'cloudCover': 61,
            'dewPoint': 14.9,
            'freezingRainIntensity': 0,
            'humidity': 41,
            'precipitationProbability': 0,
            'pressureSeaLevel': 1016.66,
            'pressureSurfaceLevel': 991.29,
            'rainIntensity': 0,
            'sleetIntensity': 0,
            'snowIntensity': 0,
            'temperature': 29.6,
            'temperatureApparent': 29.4,
            'uvHealthConcern': 3,
            'uvIndex': 7,
            'visibility': 16,
            'weatherCode': 1101,
            'windDirection': 208,
            'windGust': 4.7,
            'windSpeed': 4.1
        }
    },
    'location': {
        'lat': 40.11079406738281,
        'lon': -88.21407318115234,
        'name': 'Urbana, Champaign County, Illinois, 61801, United States',
        'type': 'postcode'
    }
}


#### Question 4: Country this location is in

The country isn't its own dictionary, only part of a dictionary containing the entire name

Workaround - reverse geocoding?

In [137]:
f'{urbana_current_data['location']['name']}'

'Urbana, Champaign County, Illinois, 61801, United States'

#### Question 5: Difference between current temp and feels like temp

In [138]:
abs(urbana_current_data['data']['values']['temperature'] - urbana_current_data['data']['values']['temperatureApparent'])

0.20000000000000284

#### Question 6: Current temp at Heathrow

In [139]:
heathrow_current_url = f'{baseurl}realtime?location=51.4680%2C%20-0.4551&apikey={API_KEY_TOMORROW}'

response = requests.get(heathrow_current_url)
heathrow_current_data = response.json()
pprint(heathrow_current_data)

{
    'data': {
        'time': '2025-06-11T18:45:00Z',
        'values': {
            'cloudBase': 10.4,
            'cloudCeiling': 11.3,
            'cloudCover': 88,
            'dewPoint': 11.5,
            'freezingRainIntensity': 0,
            'humidity': 57,
            'precipitationProbability': 0,
            'pressureSeaLevel': 1017.04,
            'pressureSurfaceLevel': 1014.5,
            'rainIntensity': 0,
            'sleetIntensity': 0,
            'snowIntensity': 0,
            'temperature': 20.2,
            'temperatureApparent': 20.2,
            'uvHealthConcern': 0,
            'uvIndex': 0,
            'visibility': 16,
            'weatherCode': 1001,
            'windDirection': 94,
            'windGust': 8.8,
            'windSpeed': 4.1
        }
    },
    'location': {'lat': 51.468, 'lon': -0.4551}
}


In [140]:
heathrow_current_data['data']['values']['temperature']

20.2

#### Question 7: URL for 3-day forecast at Heathrow

You'd use this URL, but I think the API can only give a 5-day forecast: https://api.tomorrow.io/v4/weather/forecast?location=51.4680%2C%200.4551&timesteps=1d&apikey={API_KEY_TOMORROW}

Since this gives a list of dictionaries, each for a day, we can pull the first 3 elements in the list (which are dictionaries of days) or remove the last three elements (because the length of the list is 6, I guess it's inclusive of start and end day)

In [141]:
heathrow_5day_url = f'{baseurl}forecast?location=51.4680%2C%20-0.4551&timesteps=1d&apikey={API_KEY_TOMORROW}'

response = requests.get(heathrow_5day_url)
heathrow_5day_data = response.json()
pprint(heathrow_5day_data)

{
    'timelines': {
        'daily': [
            {
                'time': '2025-06-11T05:00:00Z',
                'values': {
                    'cloudBaseAvg': 3.9,
                    'cloudBaseMax': 11.5,
                    'cloudBaseMin': 0,
                    'cloudCeilingAvg': 4.7,
                    'cloudCeilingMax': 11.9,
                    'cloudCeilingMin': 0,
                    'cloudCoverAvg': 78,
                    'cloudCoverMax': 100,
                    'cloudCoverMin': 0,
                    'dewPointAvg': 12.4,
                    'dewPointMax': 16.3,
                    'dewPointMin': 8.8,
                    'evapotranspirationAvg': 0.182,
                    'evapotranspirationMax': 0.371,
                    'evapotranspirationMin': 0.011,
                    'evapotranspirationSum': 4.361,
                    'freezingRainIntensityAvg': 0,
                    'freezingRainIntensityMax': 0,
                    'freezingRainIntensityMin': 0,
           

In [142]:
# the list we need to remove the last few elements in:
len(heathrow_5day_data['timelines']['daily'])
# answer = 6, so we need to remove the last 3 - we can just remove last item three times

heathrow_5day_data['timelines']['daily'].pop()
heathrow_5day_data['timelines']['daily'].pop()
heathrow_5day_data['timelines']['daily'].pop()

heathrow_3day_data = heathrow_5day_data['timelines']['daily']
heathrow_3day_data


[{'time': '2025-06-11T05:00:00Z',
  'values': {'cloudBaseAvg': 3.9,
   'cloudBaseMax': 11.5,
   'cloudBaseMin': 0,
   'cloudCeilingAvg': 4.7,
   'cloudCeilingMax': 11.9,
   'cloudCeilingMin': 0,
   'cloudCoverAvg': 78,
   'cloudCoverMax': 100,
   'cloudCoverMin': 0,
   'dewPointAvg': 12.4,
   'dewPointMax': 16.3,
   'dewPointMin': 8.8,
   'evapotranspirationAvg': 0.182,
   'evapotranspirationMax': 0.371,
   'evapotranspirationMin': 0.011,
   'evapotranspirationSum': 4.361,
   'freezingRainIntensityAvg': 0,
   'freezingRainIntensityMax': 0,
   'freezingRainIntensityMin': 0,
   'humidityAvg': 72,
   'humidityMax': 91,
   'humidityMin': 48,
   'iceAccumulationAvg': 0,
   'iceAccumulationLweAvg': 0,
   'iceAccumulationLweMax': 0,
   'iceAccumulationLweMin': 0,
   'iceAccumulationLweSum': 0,
   'iceAccumulationMax': 0,
   'iceAccumulationMin': 0,
   'iceAccumulationSum': 0,
   'moonriseTime': '2025-06-11T21:15:28Z',
   'moonsetTime': '2025-06-11T03:04:19Z',
   'precipitationProbabilityAvg':

#### Question 8: Print the date of each of the 3 forecast days

In [143]:
for date in heathrow_3day_data:
    print(date['time'])

2025-06-11T05:00:00Z
2025-06-12T05:00:00Z
2025-06-13T05:00:00Z


#### Question 9: Print max temp of each day

In [144]:
for date in heathrow_3day_data:
    print(f'{date['time']}: {date['values']['temperatureMax']} degrees C')

2025-06-11T05:00:00Z: 24.5 degrees C
2025-06-12T05:00:00Z: 24.8 degrees C
2025-06-13T05:00:00Z: 26.6 degrees C


#### Question 10: Print only the day with the highest max temp

In [145]:
hottestday = max(heathrow_3day_data, key=lambda date:date['values']['temperatureMax'])

f"The hottest day is {hottestday['time']}"

# the max function (like the min or sorted function) takes the list of dictionaries heathrow_3day_forecast['forecast']['forecastday'] and finds the dictionary with the largest value for the key ['day']['maxtemp_f']
# hottestday = that dictionary
# but just printing 'hottestday' here would print out the whole dictionary for that hottest day, rather than the value of the hottest day
# so to print the date of the hottest day, we print the key ['date'] for the dictionary hottestday


'The hottest day is 2025-06-13T05:00:00Z'